In [38]:
# This script can be run to create a dataset of google news headlines. 

import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup as bs4

In [79]:
#
searchTerm = "something"
count=0
url = ("https://www.google.com/search?q=" + searchTerm + "&hl=en&gl=us&authuser=0&tbs=qdr:y&tbm=nws&start=$count$&cad=h")

headlines = []
dates = []

while (count<20):
    new_url = url.replace("$count$", str(count))
    r = requests.get(new_url)
    
    soup = bs4(r.text, 'html.parser')
    
    headlines += soup.select('div.st')
    
    dates += soup.find_all('div', 'slp')
#     for i in dates:
#         dates = i.string
    
    
    
# Notification stuff...
    print("PAGE "+str(1+(count/10))+ " SCRAPED")
#     print(soup.prettify())
#     url = "https://www.google.com/search?q=ibuprofen&hl=en&gl=us&authuser=0&tbs=qdr:y&tbm=nws&start=$count$&cad=h"
    count = count+10

PAGE 1.0 SCRAPED
PAGE 2.0 SCRAPED


In [93]:
print(headlines)

[<div class="st">Houston capped off both another year in a millennium noted for teams ending championship droughts and a baseball season that had, thankfully, <b>something</b> for ...</div>, <div class="st">Now, Thirdy has a chance to do <b>something</b> that even Kiefer could not do during his own stellar career with the Blue Eagles – to sweep the elimination round of ...</div>, <div class="st">We usually try every time to eat <b>something</b> weird and exotic one of us has never eaten before. A couple of weeks ago, that was star fruit, because the kids had ...</div>, <div class="st">If you're like most people, you've been scouring the web for ideas for the perfect gift. <b>Something</b> that might not be as over-the-top as what's found in the Neiman ...</div>, <div class="st">WEDNESDAY PUZZLE — Well, look who's collaborating on crossword puzzles. Digital crossword editor Joel Fagliano and assistant crossword editor Sam ...</div>, <div class="st">We've seen <b>something</b>. We've sai

In [88]:
# def _remove_attrs(soup):
#     for tag in soup.findAll(True): 
#         tag.attrs = None
#     return soup

# def clean():
#     clean_soup = _remove_attrs(date_df)
#     return print(clean_soup)

# clean()


# date_df = pd.DataFrame({'Date':dates})


# def cleanhtml():
    
#     for i in date_df:
#       cleanr = re.compile('<.*?>')
#       cleantext = re.sub(cleanr, '', raw_html)
#       return cleantext

# cleanHtml()